In [1]:
from init import *
import time

In [2]:
def BuildModel_Simple():
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(IndependentGaussian())
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 64}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.48758983612060547],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.0002930437622126192,
  'n': 1024,
  'skewness': None,
  'stddev': 3.6327779184830433e-07},
 'skewness': [-0.007595337461680174],
 'stddev': [2.789801836013794],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.29806187748908997,
  0.49875256419181824,
  1.0983860492706299,
  2.094245195388794],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'run_time_seconds': {'mean': 14.786245346069336,
  'n': 1024,
  'skewness': None,
  'stddev': None},
 'skewness': [0.00012205391976749524,
  0.001211180235259235,
  0.0014383456436917186,
  0.0030449037440121174],
 'stddev': [2.001171827316284,
  2.8296520709991455,
  4.4733195304870605,
  6.329095840454102],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [1000, 2000, 5000, 10000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.8 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(IndependentGaussian())
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(IndependentGaussian())
model.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.2768218219280243,
  0.30182188749313354,
  0.45873183012008667,
  0.5114914178848267,
  1.0433905124664307,
  1.0934675931930542],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.21759796142578125,
  'n': 1024,
  'skewness': None,
  'stddev': 9.56650183070451e-06},
 'skewness': [0.014326480217278004,
  0.027696577832102776,
  0.022114649415016174,
  0.0045123836025595665,
  0.025011220946907997,
  -0.012584877200424671],
 'stddev': [1.9974932670593262,
  1.9750926494598389,
  2.821709632873535,
  2.8276307582855225,
  4.415787220001221,
  4.499292373657227],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [10]:
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.276822,0.019975,1.997493,0.014326
1,BrownianMotion,,1,1.0,2000,10000,0.301822,0.019751,1.975093,0.027697
2,BrownianMotion,,0,2.0,4000,10000,0.458732,0.028217,2.821710,0.022115
3,BrownianMotion,,1,2.0,4000,10000,0.511491,0.028276,2.827631,0.004512
4,BrownianMotion,,0,5.0,10000,10000,1.043391,0.044158,4.415787,0.025011
5,BrownianMotion,,1,5.0,10000,10000,1.093468,0.044993,4.499292,-0.012585


In [11]:
logging.info('The End')

2022-10-17 21:08:16,819 [INFO] The End
